In [42]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
from obspy.signal.invsim import cosine_taper
from obspy.signal.filter import highpass
from obspy.signal.trigger import classic_sta_lta, plot_trigger, trigger_onset
from filter import filter

In [43]:
data_directory = './space_apps_2024_seismic_detection/data/mars/training/data/'

cat_directory = './space_apps_2024_seismic_detection/data/mars/training/catalogs/'
cat_file = cat_directory + 'Mars_InSight_training_catalog_final.csv'
cat = pd.read_csv(cat_file)

In [44]:
# Funny

In [45]:
# How long should the short-term and long-term window be, in seconds?
sta_len = 5000
lta_len = 40000

# Play around with the on and off triggers, based on values in the characteristic function
thr_on = 2.3
thr_off = 1.5

# Sum of difference between catalogue and detected events (for testing)
diff = 0
export = []

In [ ]:
#iterate through files in directory
for row in cat.iterrows():
    filename = row[1]['filename']
    filename = filename[:-4]
    arrival_time = datetime.strptime(row[1]['time_abs(%Y-%m-%dT%H:%M:%S.%f)'],'%Y-%m-%dT%H:%M:%S.%f')
    mseed_file = f'{data_directory}{filename}.mseed'
    st = read(mseed_file)

    # This is how you get the data and the time, which is in seconds
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = filter(st)

    first_nonzero_index = np.nonzero(tr_data)[0][0]
    
    # Start time of trace (another way to get the relative arrival time using datetime)
    starttime = tr.stats.starttime.datetime
    arrival = (arrival_time - starttime).total_seconds()
    arrival

    # Initialize figure
    fig,ax = plt.subplots(1,1,figsize=(10,3))

    # Plot trace
    ax.plot(tr_times,tr_data)

    predicted_time = tr_times[first_nonzero_index]

    # Mark detection
    ax.axvline(x = arrival, color='red',label='Cat. Arrival')
    ax.axvline(x = predicted_time, color='green',label='Detect. Arrival')
    diff += arrival - predicted_time
    ax.legend(loc='upper left')

    # Add data to the export list
    export_row = {'filename': filename, 'time_abs(%Y-%m-%dT%H:%M:%S.%f)': starttime + timedelta(seconds = predicted_time), 'time_rel(sec)': tr_times[first_nonzero_index]}
    export.append(export_row)

    # Make the plot pretty
    ax.set_xlim([min(tr_times),max(tr_times)])
    ax.set_ylabel('Velocity (m/s)')
    ax.set_xlabel('Time (s)')
    ax.set_title(f'{filename}', fontweight='bold')
    plt.show()

In [ ]:
print("Avg diff: " + str(diff/len(cat)))
export_df = pd.DataFrame(export)
export_df.to_csv('output.csv', index=False)